In [1]:
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install seaborn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:


df=pd.read_csv("dataset/customer_shopping_data.csv")
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [4]:
df.columns

Index(['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity',
       'price', 'payment_method', 'invoice_date', 'shopping_mall'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [6]:
df.describe()

,age,quantity,price
count,99457.000000,99457.000000,99457.000000
mean,43.427089,3.003429,689.256321
std,14.990054,1.413025,941.184567
min,18.000000,1.000000,5.230000
25%,30.000000,2.000000,45.450000
50%,43.000000,3.000000,203.300000
75%,56.000000,4.000000,1200.320000
max,69.000000,5.000000,5250.000000


In [7]:
duplicate_rows = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicate_rows}")


Duplicate rows: 0


In [8]:
# Asegurar que la fecha sea un objeto datetime
df["invoice_date"] = pd.to_datetime(df["invoice_date"], dayfirst=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_no      99457 non-null  object        
 1   customer_id     99457 non-null  object        
 2   gender          99457 non-null  object        
 3   age             99457 non-null  int64         
 4   category        99457 non-null  object        
 5   quantity        99457 non-null  int64         
 6   price           99457 non-null  float64       
 7   payment_method  99457 non-null  object        
 8   invoice_date    99457 non-null  datetime64[ns]
 9   shopping_mall   99457 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 7.6+ MB


In [10]:
# Dimensión de Clientes (con name vacío)
df_customers = df[["customer_id", "gender", "age"]].drop_duplicates().copy()
df_customers["name"] = ''  # Agregamos 'name' vacío

In [12]:
df_customers

,customer_id,gender,age,name
0,C241288,Female,28,
1,C111565,Male,21,
2,C266599,Male,20,
3,C988172,Female,66,
4,C189076,Female,53,
...,...,...,...,...
99452,C441542,Female,45,
99453,C569580,Male,27,
99454,C103292,Male,63,
99455,C800631,Male,56,


In [13]:
# Dimensión de Categorías
df_categories = df[["category"]].drop_duplicates().reset_index(drop=True)
df_categories["category_id"] = df_categories.index + 1  # Generamos IDs únicos

In [14]:
# Dimensión de Métodos de Pago
df_payments = df[["payment_method"]].drop_duplicates().reset_index(drop=True)
df_payments["payment_id"] = df_payments.index + 1  # Generamos IDs únicos

# Dimensión de Centros Comerciales
df_malls = df[["shopping_mall"]].drop_duplicates().reset_index(drop=True)
df_malls["mall_id"] = df_malls.index + 1  # Generamos IDs únicos

# Crear una clave foránea en el DataFrame de Ventas
df_sales = df.copy()

# Asignar category_id basado en category
df_sales = df_sales.merge(df_categories, on="category", how="left")

# Asignar payment_id basado en payment_method
df_sales = df_sales.merge(df_payments, on="payment_method", how="left")

# Asignar mall_id basado en shopping_mall
df_sales = df_sales.merge(df_malls, on="shopping_mall", how="left")

# Seleccionar solo las columnas necesarias en ventas
df_sales = df_sales[["invoice_no", "customer_id", "category_id", "quantity", "price", "payment_id", "invoice_date", "mall_id"]]

In [15]:
# Mostrar los DataFrames resultantes
df_customers, df_categories, df_payments, df_malls, df_sales

(      customer_id  gender  age name
 0         C241288  Female   28     
 1         C111565    Male   21     
 2         C266599    Male   20     
 3         C988172  Female   66     
 4         C189076  Female   53     
 ...           ...     ...  ...  ...
 99452     C441542  Female   45     
 99453     C569580    Male   27     
 99454     C103292    Male   63     
 99455     C800631    Male   56     
 99456     C273973  Female   36     
 
 [99457 rows x 4 columns],
           category  category_id
 0         Clothing            1
 1            Shoes            2
 2            Books            3
 3        Cosmetics            4
 4  Food & Beverage            5
 5             Toys            6
 6       Technology            7
 7         Souvenir            8,
   payment_method  payment_id
 0    Credit Card           1
 1     Debit Card           2
 2           Cash           3,
        shopping_mall  mall_id
 0             Kanyon        1
 1     Forum Istanbul        2
 2          Met

In [16]:
df_sales

,invoice_no,customer_id,category_id,quantity,price,payment_id,invoice_date,mall_id
0,I138884,C241288,1,5,1500.40,1,2022-08-05,1
1,I317333,C111565,2,3,1800.51,2,2021-12-12,2
2,I127801,C266599,1,1,300.08,3,2021-11-09,3
3,I173702,C988172,2,5,3000.85,1,2021-05-16,4
4,I337046,C189076,3,4,60.60,3,2021-10-24,1
...,...,...,...,...,...,...,...,...
99452,I219422,C441542,8,5,58.65,1,2022-09-21,1
99453,I325143,C569580,5,2,10.46,3,2021-09-22,2
99454,I824010,C103292,5,2,10.46,2,2021-03-28,3
99455,I702964,C800631,7,4,4200.00,3,2021-03-16,5
